In [1]:
import geopandas
import pandas as pd

In [ ]:
'''
    General procedure: Read in shapefiles from different sources -> group them into the five categories: "urban areas", "mountains", "open areas", "waters" and "woods"
    -> envelope the polygons into rectangular areas -> merge neighboring areas with the same category together -> write the result to a shapefile
'''

In [2]:
def preprocessUrbanAreas():
    urbanAreas = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/sie01_f.shp"

    urbanAreasDf = geopandas.read_file(urbanAreas)
    urbanAreasDf.loc[urbanAreasDf["OBJART_TXT"] == "AX_Ortslage", "OBJART_TXT"] = "urbanAreas"

    return urbanAreasDf

In [3]:
def preprocessMountains():
    mountains = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/veg03_f.shp"

    mountainsDf = geopandas.read_file(mountains)
    mountainsDf.loc[mountainsDf["OBJART_TXT"] == "AX_UnlandVegetationsloseFlaeche", "OBJART_TXT"] = "mountains"
    
    mountainsDf.drop(mountainsDf[mountainsDf.OBJART_TXT == "AX_Gehoelz"].index, inplace=True)
    mountainsDf.drop(mountainsDf[mountainsDf.OBJART_TXT == "AX_Heide"].index, inplace=True)
    mountainsDf.drop(mountainsDf[mountainsDf.OBJART_TXT == "AX_Moor"].index, inplace=True)
    mountainsDf.drop(mountainsDf[mountainsDf.OBJART_TXT == "AX_Sumpf"].index, inplace=True)

    return mountainsDf

In [4]:
def preprocessOpenAreas():
    openAreas = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/veg01_f.shp"

    openAreasDf = geopandas.read_file(openAreas)
    openAreasDf.loc[openAreasDf["OBJART_TXT"] == "AX_Landwirtschaft", "OBJART_TXT"] = "openAreas"

    return openAreasDf

In [5]:
def preprocessWoods():
    woods1 = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/veg02_f.shp"
    woods2 = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/veg03_f.shp"

    woodsDf1 = geopandas.read_file(woods1)
    woodsDf2 = geopandas.read_file(woods2)

    woodsDf1.loc[woodsDf1["OBJART_TXT"] == "AX_Wald", "OBJART_TXT"] = "woods"

    woodsDf2.loc[woodsDf2["OBJART_TXT"] == "AX_Gehoelz", "OBJART_TXT"] = "woods"
    woodsDf2.loc[woodsDf2["OBJART_TXT"] == "AX_Heide", "OBJART_TXT"] = "woods"
    woodsDf2.loc[woodsDf2["OBJART_TXT"] == "AX_Moor", "OBJART_TXT"] = "woods"
    woodsDf2.loc[woodsDf2["OBJART_TXT"] == "AX_Sumpf", "OBJART_TXT"] = "woods"
    # this will be processed as mountains
    woodsDf2.drop(woodsDf2[woodsDf2.OBJART_TXT == "AX_UnlandVegetationsloseFlaeche"].index, inplace=True)

    return pd.concat([woodsDf1,woodsDf2])


In [6]:
def preprocessWaters():
    # waters are defined in two separate files
    waters1 = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/gew01_f.shp"
    waters2 = "zip://C:/Users/Oliver/Documents/Projekte/bkg_shape.zip/gew02_f.shp"

    # all instances of water are treated equally for our purposes -> set same column name
    watersDf1 = geopandas.read_file(waters1)
    # drop "Fliessgewaesser" i.e. rivers, because this would crash the enveloping of areas
    watersDf1.drop(watersDf1[watersDf1.OBJART_TXT == "AX_Fliessgewaesser"].index, inplace=True)
    watersDf1.loc[watersDf1["OBJART_TXT"] == "AX_Hafenbecken", "OBJART_TXT"] = "waters"
    watersDf1.loc[watersDf1["OBJART_TXT"] == "AX_StehendesGewaesser", "OBJART_TXT"] = "waters"

    watersDf2 = geopandas.read_file(waters2)
    watersDf2.loc[watersDf2["OBJART_TXT"] == "AX_Gewaessermerkmal", "OBJART_TXT"] = "waters"
    watersDf2.loc[watersDf2["OBJART_TXT"] == "AX_Polder", "OBJART_TXT"] = "waters"

    # concat dataframes and return
    return pd.concat([watersDf1,watersDf2])


In [7]:
# concat all subareas to one dataframe

finalDf = preprocessUrbanAreas()
finalDf = pd.concat([finalDf,preprocessMountains()])
finalDf = pd.concat([finalDf,preprocessOpenAreas()])
finalDf = pd.concat([finalDf,preprocessWaters()])
finalDf = pd.concat([finalDf,preprocessWoods()])

In [8]:
finalDf['OBJART_TXT'].unique()

array(['urbanAreas', 'mountains', 'openAreas', 'waters', 'woods'],
      dtype=object)

In [9]:
# envelope areas in rectangular surrounding areas
finalDf['geometry'] = finalDf['geometry'].apply(lambda x: x.envelope)

In [11]:
# merge areas by same OBJART_TXT
finalDf = finalDf.dissolve(by='OBJART_TXT').explode()

C:\Users\Oliver\AppData\Local\Temp\ipykernel_32448\2001468565.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  finalDf = finalDf.dissolve(by='OBJART_TXT').explode()


In [10]:
# write merged areas to shapefile
finalDf.to_file('combined_polygons.shp')